# Data Merging

Some observations
- We choose the 5 non-RCV cities with highest cosine similary score compared to the 7 RCV cities in CA
- There were 33 distinct cities among those 35 cities
- There are 66 non-registered voters among 21.7 million voters
- There are total of 3.9 million voters in the sampled cities
- City 'El Paso de Robles' didn't match in demographic data, so we manually searched for possible names for that city and found 'Paso Robles'
- We found 122 cases out of 312 with 0% voter turnout. This notebook is an attempt to identify correct election dates for each cities that were selected.
    
# Find four most recent election dates
    
Vote History file doesn't contain city, so we need to merge it with the DEMOGRAPHIC file in order to find out the four most recent election dates for the selected cities. 

1. Load the DEMOGRAPHIC parquet file with only registered voters from selected cities and of selected ethnicities.
     - Get the list of RCV and non-RCV cities computed based on cosine similarity in ca_similarity_search.ipynb
2. Merge the DEMOGRAPHIC with VOTE HISTORY data
3. Find 4 most recent General elections and 4 most recent Local_or_Municipal elections


In [2]:
import pandas as pd
import janitor
import gc
import numpy as np
import time

In [3]:
start_time = time.time()

In [4]:
# ------ California -------

# # change the filepath as required, we have selected the folder with the latest date
# filepath = '../data/VM2--CA--2022-04-25/'

# DEMO_filename = 'VM2--CA--2022-04-25-DEMOGRAPHIC_selected_cols.parquet'
# VOTE_filename = 'VM2--CA--2022-04-25-VOTEHISTORY_selected_cols.parquet'
# VOTE_filename_orig = 'VM2--CA--2022-04-25-VOTEHISTORY.tab'

## ------ Colorado -------

# filepath = '../data/VM2--CO--2022-04-26/'
# DEMO_filename = 'VM2--CO--2022-04-26-DEMOGRAPHIC_selected_cols.parquet'
# VOTE_filename = 'VM2--CO--2022-04-26-VOTEHISTORY_selected_cols.parquet'
# VOTE_filename_orig = 'VM2--CO--2022-04-26-VOTEHISTORY.tab'


# # ------ Maryland -------

# filepath = '../data/VM2--MD--2022-04-08/'
# DEMO_filename = 'VM2--MD--2022-04-08-DEMOGRAPHIC_selected_cols.parquet'
# VOTE_filename = 'VM2--MD--2022-04-08-VOTEHISTORY_selected_cols.parquet'
# VOTE_filename_orig = 'VM2--MD--2022-04-08-VOTEHISTORY.tab'


# # ------ Maine -------

# filepath = '../data/VM2--ME--2022-03-02/'
# DEMO_filename = 'VM2--ME--2022-03-02-DEMOGRAPHIC_selected_cols.parquet'
# VOTE_filename = 'VM2--ME--2022-03-02-VOTEHISTORY_selected_cols.parquet'
# VOTE_filename_orig = 'VM2--ME--2022-03-02-VOTEHISTORY.tab'


# # ------ Minnesota -------

# filepath = '../data/VM2--MN--2022-03-25/'
# DEMO_filename = 'VM2--MN--2022-03-25-DEMOGRAPHIC_selected_cols.parquet'
# VOTE_filename = 'VM2--MN--2022-03-25-VOTEHISTORY_selected_cols.parquet'
# VOTE_filename_orig = 'VM2--MN--2022-03-25-VOTEHISTORY.tab'


# # ------ New Mexico -------

# filepath = '../data/VM2--NM--2022-03-30/'
# DEMO_filename = 'VM2--NM--2022-03-30-DEMOGRAPHIC_selected_cols.parquet'
# VOTE_filename = 'VM2--NM--2022-03-30-VOTEHISTORY_selected_cols.parquet'
# VOTE_filename_orig = 'VM2--NM--2022-03-30-VOTEHISTORY.tab'


# # ------ Vermont -------

# filepath = '../data/VM2--VT--2022-04-20/'
# DEMO_filename = 'VM2--VT--2022-04-20-DEMOGRAPHIC_selected_cols.parquet'
# VOTE_filename = 'VM2--VT--2022-04-20-VOTEHISTORY_selected_cols.parquet'
# VOTE_filename_orig = 'VM2--VT--2022-04-20-VOTEHISTORY.tab'


# # ------ Utah -------

filepath = '../data/VM2--UT--2022-03-30/'
DEMO_filename = 'VM2--UT--2022-03-30-DEMOGRAPHIC_selected_cols.parquet'
VOTE_filename = 'VM2--UT--2022-03-30-VOTEHISTORY_selected_cols.parquet'
VOTE_filename_orig = 'VM2--UT--2022-03-30-VOTEHISTORY.tab'


## 1.  Load new Demographic Data

1. use parquet file that was created in by Reduce_to_parquet.ipynb
2. filter the data based on the list of cities found in ca_similarity_search.ipynb


In [5]:
def combine_cities_list(RCV_list, NonRCV_list):

    print("total number of cities:", len(RCV_list))

    print("number of distinct cities:", len(set(NonRCV_list)))

    print("name of cities that were duplicated:", set([x for x in NonRCV_list if NonRCV_list.count(x) > 1]))

    combined_cityName = RCV_list+list(set(NonRCV_list))
    print("number of distinct RCV and sampled nonRCV cities:", len(combined_cityName))
    return combined_cityName

In [7]:
# 1. List of RCV and non-RCV cities 

# # ------ California -------

# RCV_cities_CA = ['San Francisco',
#  'Oakland',
#  'Berkeley',
#  'San Leandro',
#  'Palm Desert',
#  'Eureka',
#  'Albany']

# sampled_nonRCV_cities_CA = ['Fresno',
#  'San Diego',
#  'Sacramento',
#  'Riverside',
#  'San Jose',
#  'Santa Ana',
#  'Anaheim',
#  'Santa Rosa',
#  'Merced',
#  'Santa Clarita',
#  'Alhambra',
#  'Davis',
#  'Montebello',
#  'Burbank',
#  'Huntington Park',
#  'Bellflower',
#  'Watsonville',
#  'Gilroy',
#  'Whittier',
#  'Lynwood',
#  'Lakewood',
#  'Pico Rivera',
#  'Lake Forest',
#  'Livermore',
#  'Chino Hills',
#  'Paramount',
#  'El Paso de Robles',
#  'Pico Rivera',
#  'Buena Park',
#  'Whittier',
#  'Calabasas',
#  'Carpinteria',
#  'Morro Bay',
#  'San Carlos',
#  'Solvang']

# combined_sampled_cityName = combine_cities_list(RCV_list= RCV_cities_CA, NonRCV_list = sampled_nonRCV_cities_CA)
# # ---------------------



# ------ Utah -------

RCV_cities_UT = ['Salt Lake City', 'Sandy', 'Lehi', 'Millcreek', 
                 'Draper', 'Riverton',  'Cottonwood Heights', 
                 'Springville', 'Midvale', 'Magna', 'South Salt Lake', 
                 'Payson', 'Bluffdale']

sampled_nonRCV_cities_UT = ['Ogden', 'Provo', 'West Valley City', 
                            'Logan', 'St. George', 'Taylorsville', 
                            'Layton', 'Orem', 'South Jordan', 'Murray', 
                            'South Jordan', 'Clearfield', 'Spanish Fork', 
                            'Tooele', 'Kearns', 'Cedar City', 'Murray', 
                            'Bountiful',  'South Jordan', 'Pleasant Grove', 
                            'Vernal', 'Hurricane', 'Herriman', 'American Fork', 
                            'Washington', 'Eagle Mountain', 'Brigham City', 
                            'American Fork', 'Herriman', 'Spanish Fork', 
                            'Washington', 'Heber', 'Hurricane', 'Vernal', 
                            'Holladay', 'Pleasant Grove', 'American Fork', 
                            'Herriman', 'Eagle Mountain', 'Vernal', 
                            'Bountiful', 'Pleasant Grove', 'Washington', 
                            'South Jordan', 'Vernal', 'Tooele', 
                            'Spanish Fork', 'Clearfield', 'Kearns', 
                            'Eagle Mountain', 'Washington', 'Bountiful', 
                            'Pleasant Grove', 'Hurricane', 'Cedar City', 
                            'Saratoga Springs', 'Kaysville', 'Brigham City', 
                            'North Salt Lake', 'American Fork', 'Highland', 
                            'Lindon', 'Alpine', 'West Haven', 'North Logan']


combined_sampled_cityName = combine_cities_list(RCV_list= RCV_cities_UT, NonRCV_list = sampled_nonRCV_cities_UT)
# ---------------------


total number of cities: 13
number of distinct cities: 34
name of cities that were duplicated: {'Bountiful', 'Hurricane', 'Washington', 'Spanish Fork', 'Pleasant Grove', 'Cedar City', 'Vernal', 'Herriman', 'Eagle Mountain', 'Murray', 'Kearns', 'Brigham City', 'Tooele', 'South Jordan', 'Clearfield', 'American Fork'}
number of distinct RCV and sampled nonRCV cities: 47


In [8]:
def read_DEMOGRAPHIC():
    df_demographic = pd.read_parquet(f'{filepath}{DEMO_filename}')
    print("Total number of unique cities:", df_demographic.Residence_Addresses_City.nunique())
    print("Total number of unique voters:", df_demographic.LALVOTERID.nunique())
    print("Count of non-registered voters:", len(df_demographic[df_demographic['Voters_OfficialRegDate'].isnull()]))
    
    print("Number of expected cities:", len(combined_sampled_cityName))
    missing_cities = [city for city in combined_sampled_cityName if city not in df_demographic['Residence_Addresses_City'].unique()]
    if len(missing_cities) > 0:
        print("number of cities not found in demographic data:", len(missing_cities))
        print(missing_cities)
        
    return df_demographic
        
state_demographic = read_DEMOGRAPHIC()

Total number of unique cities: 389
Total number of unique voters: 1442583
Count of non-registered voters: 467
Number of expected cities: 47
number of cities not found in demographic data: 1
['St. George']


In [9]:
state_demographic.head(5)

,LALVOTERID,Residence_Addresses_City,Voters_Gender,Voters_Age,Parties_Description,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate,County,CommercialData_Education,CommercialData_EstimatedHHIncomeAmount,FECDonors_NumberOfDonations,FECDonors_TotalDonationsAmount
0,LALUT168588809,Greenville,F,81,Republican,European,09/24/2019,BEAVER,HS Diploma - Likely,$37000,None,None
1,LALUT169183714,Beaver,F,77,Republican,European,06/22/2010,BEAVER,None,$55155,None,None
2,LALUT169183783,Beaver,M,79,Republican,European,06/25/2002,BEAVER,Bach Degree - Extremely Likely,$48000,None,None
3,LALUT169165557,Beaver,F,79,Republican,European,06/25/2002,BEAVER,Some College -Extremely Likely,$48000,None,None
4,LALUT169173676,Beaver,M,75,Republican,European,06/25/2002,BEAVER,HS Diploma - Extremely Likely,$41000,None,None


In [10]:
# # ----- California ----- 
# combined_sampled_cityName = list(map(lambda x: x.replace('El Paso de Robles', 'Paso Robles'), combined_sampled_cityName))
# print("number of expected cities:", len(combined_sampled_cityName))
# # ----------------------

# ----- Utah ----- 
combined_sampled_cityName = list(map(lambda x: x.replace('St. George', 'Saint George'), combined_sampled_cityName))
print("number of expected cities:", len(combined_sampled_cityName))
# ----------------------


number of expected cities: 47


In [11]:
# 2. filter DEMOGRAPHIC data based on the list of cities, ethnicities and registered voters

selected_ethnicities = ['European', 'Likely African-American','Hispanic and Portuguese', 'East and South Asian']

def filter_demo(df, list_cityNames):
    filtered_df = df[df['Residence_Addresses_City'].isin(list_cityNames) &
            df['EthnicGroups_EthnicGroup1Desc'].isin(selected_ethnicities) &
            df['Voters_OfficialRegDate'].notnull()][['LALVOTERID', 'Residence_Addresses_City']]
    
    print(filtered_df.shape)
    print("number of unique cities:", filtered_df.Residence_Addresses_City.nunique())
    
    return filtered_df

state_demographic_subset = filter_demo(df = state_demographic, list_cityNames = combined_sampled_cityName)
state_demographic_subset.head()

(887909, 2)
number of unique cities: 47


,LALVOTERID,Residence_Addresses_City
3133,LALUT621510668,Brigham City
3137,LALUT168437912,Brigham City
3138,LALUT169786507,Brigham City
3142,LALUT596605091,Brigham City
3143,LALUT168453641,Brigham City


In [12]:
gc.collect()

478

In [13]:
del state_demographic
gc.collect()

0

## 2. Merge VoteHistory with DEMOGRAPHIC Data 
1. Kernel died when trying to load all General and Local, so we load the two types of elections separately
    1. Load the original data in order to get the complete list of all possible columns containing "General" and "Local_or_Municipal" (need only one row)
    2. create two lists with the column names one for each type of election
2. merge only city (`Residence_Addresses_City`) from DEMOGRAPHIC file to VOTE HISTORY reduce computation time

In [14]:
# 1.A select only one rows to find the column names that are General and Local_or_Municipal elections
# need to use original tab file because pandas' read_parquet doesn't support nrows

state_voterhistory_cols = pd.read_csv(f'{filepath}{VOTE_filename_orig}',
                                 sep='\t', dtype=str, encoding='unicode_escape',
                                nrows=1)

In [15]:
# 1.B select only voter ID and columns with General or Local_or_Municipal election dates
def get_elec_cols(df, string):
    return [col for col in df.columns if col.startswith(string)]
    
GE_cols = get_elec_cols(state_voterhistory_cols, 'General')
print("total number of General election dates", len(GE_cols))

LM_cols = get_elec_cols(state_voterhistory_cols, 'Local_or_Municipal')
print("total number of Local or Municipal election dates", len(LM_cols))


total number of General election dates 17
total number of Local or Municipal election dates 49


In [16]:
del state_voterhistory_cols
gc.collect()

0

In [17]:
# 2. read the VOTEHISTORY parquet file and merge the city from DEMOGRAPHIC file 
df_voterhistory_LM = pd.merge(state_demographic_subset, 
                              pd.read_parquet(f'{filepath}{VOTE_filename}', columns =['LALVOTERID'] +LM_cols), 
                               how='inner', on = 'LALVOTERID') 
df_voterhistory_LM

,LALVOTERID,Residence_Addresses_City,Local_or_Municipal_2021_06_22,Local_or_Municipal_2020_09_08,Local_or_Municipal_2020_05_12,Local_or_Municipal_2017_06_27,Local_or_Municipal_2016_07_28,Local_or_Municipal_2013_06_25,Local_or_Municipal_2012_06_19,Local_or_Municipal_2011_10_24,...,Local_or_Municipal_1996_05_07,Local_or_Municipal_1995_09_12,Local_or_Municipal_1995_08_28,Local_or_Municipal_1995_06_27,Local_or_Municipal_1995_05_23,Local_or_Municipal_1995_02_07,Local_or_Municipal_1994_08_09,Local_or_Municipal_1994_02_27,Local_or_Municipal_1993_08_31,Local_or_Municipal_1993_02_27
0,LALUT621510668,Brigham City,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,LALUT168437912,Brigham City,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,LALUT169786507,Brigham City,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,LALUT596605091,Brigham City,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,LALUT168453641,Brigham City,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887904,LALUT169831299,Ogden,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
887905,LALUT169347305,Ogden,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
887906,LALUT169298302,Ogden,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
887907,LALUT169833955,West Haven,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [18]:
# 2.1. reduce number of columns by removing columns if all rows are None

def remove_all_None(df, selected_cols):
    print("-"*20, "\nBefore filtering Vote and Demographic\n", "-"*20)
    print("Total number of General election dates", len(get_elec_cols(df, 'General')))
    print("Total number of Local or Municipal election dates", len(get_elec_cols(df, 'Local_or_Municipal')))
    print("\n")

    # reduce the search space with this step
    cols_all_None = [col for col in selected_cols if len(df[col].value_counts()) == 0]
    print("number of columns with all None:", len(cols_all_None))

    df = df.drop(columns = cols_all_None)
    print("-"*20, "\nAfter removing dates with all None\n", "-"*20)
    print("Total number of General election dates", len(get_elec_cols(df, 'General')))
    print("Total number of Local or Municipal election dates", len(get_elec_cols(df, 'Local_or_Municipal')))

    gc.collect()
    return df

In [19]:
# ------ Utah ------
df_voterhistory_LM = remove_all_None(df = df_voterhistory_LM, selected_cols = LM_cols)

-------------------- 
Before filtering Vote and Demographic
 --------------------
Total number of General election dates 0
Total number of Local or Municipal election dates 49


number of columns with all None: 3
-------------------- 
After removing dates with all None
 --------------------
Total number of General election dates 0
Total number of Local or Municipal election dates 46


In [17]:
# ------ California ------
df_voterhistory_LM = remove_all_None(df = df_voterhistory_LM, selected_cols = LM_cols)

-------------------- 
Before filtering Vote and Demographic
 --------------------
Total number of General election dates 0
Total number of Local or Municipal election dates 131


number of columns with all None: 16
-------------------- 
After removing dates with all None
 --------------------
Total number of General election dates 0
Total number of Local or Municipal election dates 115



## 3. Find 4 most recent General elections and 4 most recent Local_or_Municipal elections
1. reduce number of columns by removing columns if all rows are None
2. run a loop to only keep election date columns that are associated with the chosen subset of cities. 
3. create two dictionaries
    1. `init_city_cnt_dates_{LM|GE}` will count the number of election dates for each city
        - for each city if the count of election dates reaches 4 then stop checking more dates for that city (to do this we will remove the city from the `init_city_cnt_dates_{LM|GE}` dictionary) 
    2. `init_city_4_dates_{LM|GE}` will keep track of the cities and their election dates
        - for a given election date if at least one voter has "Y" then proceed to find which cities took part on that date
        - for each city in `init_city_cnt_dates_{LM|GE}` if the city is also present in the dataframe (i.e. the vote "Y" is counted) then increment the count by 1 in `init_city_cnt_dates_{LM|GE}` and also add the date to `init_city_4_dates_{LM|GE}`

    

In [20]:
def get_list_elec_dates(df, date_cols, list_city_cnt_dates, list_city_4_dates):
    for date_col in date_cols:
        cnt_df = df[df[date_col] == 'Y'][[date_col, 'Residence_Addresses_City']].groupby('Residence_Addresses_City').count()
        
        # If no rows found then none of the city had election held on that date 
        # assuming that at least one voter will present on an election date
        
        if len(cnt_df) > 1 and len(list_city_cnt_dates) > 0:
            
            # for the selected date check which cities held the election on that date
            for city in list(list_city_cnt_dates.keys()): 
                # first check if the city is present in list_city_cnt_dates, 
                # not being present means we have already found the dates so no need to check 
                if city in cnt_df.index:
                    # second check if the city is present in the dataframe with "Y"
                    # not being present means the date is not the election date for this city
                    list_city_cnt_dates[city] += 1
                    list_city_4_dates[city].append(date_col)                
                    if list_city_cnt_dates[city] == 4:
                        # remove the city from dictionary list_city_cnt_dates so that we know when to stop checking for more dates
                        del list_city_cnt_dates[city]
                        
        elif len(cnt_df) == 0:
            print("No cities found for ", date_col)
            
        elif len(list_city_cnt_dates) == 0:
            # means all 4 dates for all cities found since we removed cities every time 4 dates were found
            break
            
    return list_city_cnt_dates, list_city_4_dates

In [21]:
init_city_cnt_dates_LM = {key: 0 for key in df_voterhistory_LM['Residence_Addresses_City']}
init_city_4_dates_LM = {key: [] for key in df_voterhistory_LM['Residence_Addresses_City']}

#need to recompute the list of election dates because some columns were removed in the previous step
LM_cols = get_elec_cols(df_voterhistory_LM, 'Local_or_Municipal')

list_city_cnt_dates_LM, list_city_4_dates_LM = get_list_elec_dates(df_voterhistory_LM, 
                                                                   LM_cols, 
                                                                   init_city_cnt_dates_LM, 
                                                                   init_city_4_dates_LM)

if len(list_city_cnt_dates_LM) == 0:
    print("\nAll local and municipal election dates found!")
else:
    print("\nNeed to find more local and municipal election dates!!!")
print(list_city_cnt_dates_LM)
list_city_4_dates_LM


Need to find more local and municipal election dates!!!
{'Kearns': 3, 'South Salt Lake': 0, 'Heber': 2}


{'Brigham City': ['Local_or_Municipal_2017_06_27',
  'Local_or_Municipal_2013_06_25',
  'Local_or_Municipal_2009_06_23',
  'Local_or_Municipal_2007_06_26'],
 'Logan': ['Local_or_Municipal_2013_06_25',
  'Local_or_Municipal_2009_06_23',
  'Local_or_Municipal_2007_06_26',
  'Local_or_Municipal_2005_06_28'],
 'North Logan': ['Local_or_Municipal_2013_06_25',
  'Local_or_Municipal_2007_06_26',
  'Local_or_Municipal_2004_05_04',
  'Local_or_Municipal_2004_02_03'],
 'Kaysville': ['Local_or_Municipal_2013_06_25',
  'Local_or_Municipal_2011_06_28',
  'Local_or_Municipal_2009_06_23',
  'Local_or_Municipal_2007_06_26'],
 'Layton': ['Local_or_Municipal_2017_06_27',
  'Local_or_Municipal_2013_06_25',
  'Local_or_Municipal_2009_06_23',
  'Local_or_Municipal_2007_06_26'],
 'Clearfield': ['Local_or_Municipal_2013_06_25',
  'Local_or_Municipal_2007_06_26',
  'Local_or_Municipal_2005_06_28',
  'Local_or_Municipal_2004_08_03'],
 'Bountiful': ['Local_or_Municipal_2013_06_25',
  'Local_or_Municipal_2012_06

In [22]:
del df_voterhistory_LM
gc.collect()

0

## Redo 2.2 and all steps of 3 on General election

In [23]:
# 2. read the VOTEHISTORY parquet file and merge the city from DEMOGRAPHIC file 
df_voterhistory_GE = pd.merge(state_demographic_subset, 
                              pd.read_parquet(f'{filepath}{VOTE_filename}', columns =['LALVOTERID'] + GE_cols), 
                               how='inner', on = 'LALVOTERID') 
df_voterhistory_GE

,LALVOTERID,Residence_Addresses_City,General_2020_11_03,General_2018_11_06,General_2016_11_08,General_2014_11_04,General_2012_11_06,General_2010_11_02,General_2008_11_04,General_2006_11_07,General_2004_11_02,General_2002_11_05,General_2000_11_07,General_1998_11_03,General_1996_11_05,General_1994_11_08,General_1992_11_03,General_1990_11_06,General_1988_11_08
0,LALUT621510668,Brigham City,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,LALUT168437912,Brigham City,None,None,Y,None,Y,None,None,Y,Y,Y,None,None,None,None,None,None,None
2,LALUT169786507,Brigham City,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,None,Y,Y,None,None,None
3,LALUT596605091,Brigham City,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,LALUT168453641,Brigham City,Y,None,None,None,Y,None,Y,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887904,LALUT169831299,Ogden,Y,Y,None,None,Y,None,Y,None,Y,None,None,None,None,None,None,None,None
887905,LALUT169347305,Ogden,Y,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
887906,LALUT169298302,Ogden,None,Y,None,None,None,None,None,Y,Y,None,None,None,None,None,None,None,None
887907,LALUT169833955,West Haven,Y,Y,Y,Y,Y,Y,Y,None,Y,Y,None,None,None,None,None,None,None


In [24]:
df_voterhistory_GE = remove_all_None(df_voterhistory_GE, GE_cols)

init_city_cnt_dates_GE = {key: 0 for key in df_voterhistory_GE['Residence_Addresses_City']}
init_city_4_dates_GE = {key: [] for key in df_voterhistory_GE['Residence_Addresses_City']}

GE_cols = get_elec_cols(df_voterhistory_GE, 'General')

list_city_cnt_dates_GE, list_city_4_dates_GE = get_list_elec_dates(df_voterhistory_GE, 
                                                                   GE_cols, 
                                                                   init_city_cnt_dates_GE, 
                                                                   init_city_4_dates_GE)

if len(list_city_cnt_dates_GE) == 0:
    print("\nAll local and municipal election dates found!")
else:
    print("\nNeed to find more local and municipal election dates!!!")
print(list_city_cnt_dates_GE)
list_city_4_dates_GE

-------------------- 
Before filtering Vote and Demographic
 --------------------
Total number of General election dates 17
Total number of Local or Municipal election dates 0


number of columns with all None: 0
-------------------- 
After removing dates with all None
 --------------------
Total number of General election dates 17
Total number of Local or Municipal election dates 0

All local and municipal election dates found!
{}


{'Brigham City': ['General_2020_11_03',
  'General_2018_11_06',
  'General_2016_11_08',
  'General_2014_11_04'],
 'Logan': ['General_2020_11_03',
  'General_2018_11_06',
  'General_2016_11_08',
  'General_2014_11_04'],
 'North Logan': ['General_2020_11_03',
  'General_2018_11_06',
  'General_2016_11_08',
  'General_2014_11_04'],
 'Kaysville': ['General_2020_11_03',
  'General_2018_11_06',
  'General_2016_11_08',
  'General_2014_11_04'],
 'Layton': ['General_2020_11_03',
  'General_2018_11_06',
  'General_2016_11_08',
  'General_2014_11_04'],
 'Clearfield': ['General_2020_11_03',
  'General_2018_11_06',
  'General_2016_11_08',
  'General_2014_11_04'],
 'Bountiful': ['General_2020_11_03',
  'General_2018_11_06',
  'General_2016_11_08',
  'General_2014_11_04'],
 'Ogden': ['General_2020_11_03',
  'General_2018_11_06',
  'General_2016_11_08',
  'General_2014_11_04'],
 'North Salt Lake': ['General_2020_11_03',
  'General_2018_11_06',
  'General_2016_11_08',
  'General_2014_11_04'],
 'Draper'

In [25]:
GE_dates_df = pd.DataFrame(list_city_4_dates_GE.items(), columns=['city', 'GE_dates'])
LM_dates_df = pd.DataFrame(list_city_4_dates_LM.items(), columns=['city', 'LM_dates'])
GE_LM_dates_df = GE_dates_df.merge(LM_dates_df, on = "city")
GE_LM_dates_df.head()

,city,GE_dates,LM_dates
0,Brigham City,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2017_06_27, Local_or_Munic..."
1,Logan,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2013_06_25, Local_or_Munic..."
2,North Logan,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2013_06_25, Local_or_Munic..."
3,Kaysville,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2013_06_25, Local_or_Munic..."
4,Layton,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2017_06_27, Local_or_Munic..."


In [26]:
# # save in parquet format

# ------ California ------
# GE_LM_dates_df.to_parquet(f'{filepath}GE_LM_dates_per_city_CA.parquet')

# ------ Utah ------
GE_LM_dates_df.to_parquet(f'{filepath}GE_LM_dates_per_city_UT.parquet')

In [27]:
end_time = time.time()
print("Time take to run this notebook in seconds: ", end_time - start_time)

Time take to run this notebook in seconds:  427.5217730998993


In [28]:
del df_voterhistory_GE
gc.collect()

0